In [1]:
import numpy as np
import pandas as pd
import requests
import os
from dotenv import load_dotenv
import json
from datetime import datetime

In [2]:
# Load the API key from the .env file
load_dotenv()
api_key = os.getenv("API_KEY")

# Fetch the odds data
sport_key = "basketball_ncaab"
regions = "eu"  # Regions: us, us2, uk, eu, au
markets = "h2h"  # Betting markets
odds_format = "decimal"  # Odds formats: decimal, american

url = f"https://api.the-odds-api.com/v4/sports/{sport_key}/odds"
params = {
    "apiKey": api_key,
    "regions": regions,
    "markets": markets,
    "oddsFormat": odds_format
}

response = requests.get(url, params=params)

if response.status_code == 200:
    data = response.json()
    
    # Add timestamp and parameters to the data
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    for game in data:
        game["timestamp"] = timestamp
    
    # Generate a unique filename using a timestamp
    filename = f"odds_data_{datetime.now().strftime('%Y%m%d%H%M%S')}_{sport_key}_{regions}_{markets}_{odds_format}.json"
    
    # Save the JSON response to a file
    with open("json_requests/"+filename, "w") as json_file:
        json.dump(data, json_file, indent=4)
    
else:
    print(f"Error: {response.status_code} - {response.text}")

In [5]:
# Prepare list for flattened data entries
flattened_data = []
for game in data:
    base_info = {
        "id": game["id"],
        "sport_key": game["sport_key"],
        "sport_title": game["sport_title"],
        "commence_time": game["commence_time"],
        "home_team": game["home_team"],
        "away_team": game["away_team"],
        "timestamp": game["timestamp"]
    }
    
    # Process each bookmaker and its markets
    for bookmaker in game["bookmakers"]:
        bookmaker_key = bookmaker["key"]
        for market in bookmaker["markets"]:
            market_key = market["key"]
            
            entry = base_info.copy()
            for outcome in market["outcomes"]:
                team = "home" if outcome["name"] == game["home_team"] else "away"
                # Generalized column name based on market type and team
                column_name = f"{bookmaker_key}_{team}_{market_key}_odds"
                entry[column_name] = outcome["price"]
            
            # Add the entry to flattened_data
            flattened_data.append(entry)

# Convert the flattened data into a DataFrame
df = pd.DataFrame(flattened_data).drop_duplicates()

# Collapse rows for each fixture by using `groupby` and `agg`
df = df.groupby(
    ["id", "sport_key", "sport_title", "commence_time", "home_team", "away_team", "timestamp"],
    as_index=False
).agg("first")  # Use "first" to take the first non-null value in each group

df

,id,sport_key,sport_title,commence_time,home_team,away_team,timestamp,sport888_away_h2h_odds,sport888_home_h2h_odds,mybookieag_away_h2h_odds,...,betfair_ex_eu_away_h2h_odds,betfair_ex_eu_home_h2h_odds,betfair_ex_eu_away_h2h_lay_odds,betfair_ex_eu_home_h2h_lay_odds,marathonbet_away_h2h_odds,marathonbet_home_h2h_odds,gtbets_home_h2h_odds,gtbets_away_h2h_odds,nordicbet_home_h2h_odds,nordicbet_away_h2h_odds
0,0050b8419f91f1e2b59330e6bd03883d,basketball_ncaab,NCAAB,2025-01-18T22:00:00Z,Texas State Bobcats,Southern Miss Golden Eagles,2025-01-18 10:38:49,4.20,1.22,3.82,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0062015412f1928f15ba6b92059dcc9a,basketball_ncaab,NCAAB,2025-01-18T19:00:00Z,Miami Hurricanes,SMU Mustangs,2025-01-18 10:38:49,1.45,2.75,1.43,...,NaN,NaN,NaN,NaN,1.44,2.73,NaN,NaN,2.68,1.44
2,008f4cf1eec04d0fa2c3eaae645154a7,basketball_ncaab,NCAAB,2025-01-19T00:00:00Z,Wichita St Shockers,East Carolina Pirates,2025-01-18 10:38:49,2.40,1.57,2.40,...,NaN,NaN,NaN,NaN,NaN,NaN,1.61,2.40,NaN,NaN
3,01773e4bca697eb37057f1fe8b61f56c,basketball_ncaab,NCAAB,2025-01-19T00:00:00Z,Marist Red Foxes,Rider Broncs,2025-01-18 10:38:49,4.10,1.22,4.02,...,NaN,NaN,NaN,NaN,NaN,NaN,1.25,4.10,NaN,NaN
4,01adcc73522a35eac59b7166b2fe57bb,basketball_ncaab,NCAAB,2025-01-18T19:00:00Z,Northern Kentucky Norse,Wright St Raiders,2025-01-18 10:38:49,2.05,1.75,2.08,...,NaN,NaN,NaN,NaN,2.09,1.72,1.80,2.05,1.72,2.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,f720932b3e67b0fe5803cd3684c2bd99,basketball_ncaab,NCAAB,2025-01-19T00:00:00Z,High Point Panthers,Presbyterian Blue Hose,2025-01-18 10:38:49,7.00,1.10,5.75,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
142,fa8ebbec3138d421cfb48b9de3f98c26,basketball_ncaab,NCAAB,2025-01-18T22:00:00Z,Jacksonville St Gamecocks,Sam Houston St Bearkats,2025-01-18 10:38:49,2.15,1.70,2.14,...,NaN,NaN,NaN,NaN,NaN,NaN,1.71,2.20,NaN,NaN
143,fb4f7e1187517d4bf23b99b096ba3c56,basketball_ncaab,NCAAB,2025-01-18T20:30:00Z,Coastal Carolina Chanticleers,Marshall Thundering Herd,2025-01-18 10:38:49,1.57,2.40,NaN,...,NaN,NaN,NaN,NaN,1.58,2.34,2.45,1.59,2.30,1.58
144,fdd369131c41c9e6d31a9e7e5b457da5,basketball_ncaab,NCAAB,2025-01-18T19:00:00Z,Northern Illinois Huskies,Central Michigan Chippewas,2025-01-18 10:38:49,NaN,NaN,1.21,...,NaN,NaN,NaN,NaN,1.23,4.05,NaN,NaN,4.35,1.19
